In [6]:
%load_ext autoreload
%autoreload 2

import numpy as np
from pathlib import Path
from PIL import Image
import torchvision.transforms as transforms
import torch.utils.data as data

from dataset import ImageDataset
import networks
from utils import train

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
hres_size = (512, 512)
scale_factor = 4

img_paths = []
for dir in Path('synthetic_images').glob('**/*'):
    img_paths.extend(dir.glob('*.png'))

dataset = ImageDataset(img_paths, hres_size, scale_factor, transforms.Compose((transforms.ToTensor(), transforms.Normalize(mean=0.5, std=0.5))))

In [12]:
batch_size = 16

train_set, test_set = data.random_split(dataset, [0.5, 0.5])

trainloader = data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
testloader = data.DataLoader(test_set, batch_size=batch_size, shuffle=True)

In [ ]:
class Args():
    device = "cuda"
    lr_net = 0.005
    epoch_eval_train = 20

args = Args()

In [13]:
net = networks.SRCNN(3, scale_factor)
train(0, net, trainloader, testloader, args)

KeyboardInterrupt: 